# Algoritmos de optimización - Reto 3

Nombre: Jair Francisco Flores Farfan<br>
Github: https://github.com/jfloresf17/miar-viu/Algoritmos/Reto_3

## Colonia de Hormigas para el problema del agente viajero (42 ciudades)

In [1]:
import numpy as np
import tsplib95
import urllib
import gzip

In [2]:
## Descomprimir el archivo
file = 'swiss42.tsp'
urllib.request.urlretrieve("http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/swiss42.tsp.gz", file + '.gz')

with gzip.open(file + '.gz', 'rb') as f:
    data = f.read()
    with open(file, 'wb') as f:
        f.write(data)

## Resolviendo el TSP con la Colonia de Hormigas (ACO)

El **Problema del Viajante (TSP)** consiste en encontrar el camino más corto que visite todos los nodos exactamente una vez y regrese al punto de inicio. La **Colonia de Hormigas (ACO)** es un algoritmo bioinspirado que simula el comportamiento de las hormigas al buscar la ruta óptima mediante la **colocación y evaporación de feromonas**.

Se definen inicialmente dos funciones que nos sirven para determinar la distancia entre dos nodos (`distancia`) y el coste de la solución (`distancia_total`), distancia total recorrida por la hormiga.

In [ ]:
## Devuelve la distancia entre dos nodos
def distancia(a: int,b: int, problem: tsplib95.models.Problem) -> float:
  """
  Devuelve la distancia entre dos nodos

  Args:
      a (int): Nodo de inicio
      b (int): Nodo de destino
      problem (tsplib95.models.Problem): Problema del agente viajero

  Returns:
      float: Distancia entre los dos nodos
  """
  return problem.get_weight(a,b)

## Devuelve la distancia total de una trayectoria/solucion(lista de nodos)
def distancia_total(solucion: list, problem: tsplib95.models.Problem) -> float:
  """
  Devuelve la distancia total de una trayectoria/solucion(lista de nodos)

  Args:
      solucion (list): Lista de nodos que representan una solucion
      problem (tsplib95.models.Problem): Problema del agente viajero

  Returns:
      float: Distancia total de la solucion
  """
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)

  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)

### **Función `add_nodo()`**
La función Add_Nodo selecciona al azar un nodo con probabilidad uniforme definida por la siguiente fórmula:

$p^k_{ij}(t) = \frac{[\tau_{ij}(t)]^\alpha[\nu_{ij}]^\beta}{\sum_{l\in J^k_i} [\tau_{il}(t)]^\alpha[\nu_{il}]^\beta}$, si $j \in J^k_i$

$p^k_{ij}(t) = 0$, si $j \notin J^k_i$

Donde:
- $p^k_{ij}(t)$ → Probabilidad de ir del nodo $i$ al nodo $j$ en la iteración $t$.
- $[\tau_{ij}(t)]$ → Cantidad de feromonas en la arista $(i, j)$ en la iteración $t$.
- $[\nu_{ij}]$ → Heurística (distancia inversa) entre los nodos $i$ y $j$.
- $J^k_i$ → Conjunto de nodos no visitados por la hormiga $k$.

In [4]:
def add_nodo(actual: int, nodos_disponibles: list, nodos: list, 
             feromonas: np.ndarray, distancias: np.ndarray, 
             alpha: float, beta: float) -> int:
    """
    Selecciona el siguiente nodo basado en feromonas y heurística de distancia.

    La probabilidad de seleccionar un nodo se calcula con la ecuación:
    p_{ij} = (\tau_{ij}^\alpha) * (\eta_{ij}^\beta) / \sum (\tau_{ij}^\alpha) * (\eta_{ij}^\beta)

    Donde:
    - p_{ij} es la probabilidad de ir del nodo i al nodo j.
    - \tau_{ij} es la cantidad de feromonas entre los nodos i y j.
    - \alpha es el peso de la influencia de las feromonas.
    - \eta_{ij} es la visibilidad entre los nodos i y j (inversa de la distancia).
    - \beta es el peso de la influencia de la heurística de distancia.
       
    Args:
    actual (int): Nodo actual donde se encuentra la hormiga.
    nodos_disponibles (list): Lista de nodos aún no visitados.
    nodos (list): Lista de nodos en el problema TSP.
    feromonas (ndarray): Matriz de feromonas entre nodos.
    distancias (ndarray): Matriz de distancias entre nodos.
    alpha (float): Peso de la influencia de las feromonas.
    beta (float): Peso de la influencia de la heurística de distancia.
    
    Returns:
    int: El siguiente nodo seleccionado.
    """
    if not nodos_disponibles:
        return None  # No hay nodos disponibles
    
    # Obtener índices de los nodos disponibles
    indices_disponibles = [nodos.index(j) for j in nodos_disponibles]
    actual_idx = nodos.index(actual)
    
    # Calcular visibilidad (inversa de la distancia)
    visibilidad = np.zeros(len(indices_disponibles))
    for idx, j in enumerate(indices_disponibles):
        visibilidad[idx] = 1 / distancias[actual_idx, j] if distancias[actual_idx, j] > 0 else 0
    
    # Calcular valores de probabilidad con la ecuación
    numeradores = (feromonas[actual_idx, indices_disponibles] ** alpha) * (visibilidad ** beta)
    denominador = np.sum(numeradores)
    
    # Calcular probabilidades normalizadas
    probabilidades = numeradores / denominador if denominador > 0 \
                     else np.ones(len(indices_disponibles)) / len(indices_disponibles)
    
    # Seleccionar el siguiente nodo basado en las probabilidades
    siguiente_nodo = np.random.choice(nodos_disponibles, p=probabilidades)
    return siguiente_nodo

Para solucionar el problema del agente viajero, se implementa el algoritmo de la colonia de hormigas (ACO) con las siguientes etapas:

#### **1. Inicialización**
- Inicializar la **matriz de feromonas** con valores pequeños `(1/num_nodos).`
- Configurar los parámetros:
  - $ \alpha $ → Influencia de las feromonas.
  - $ \beta $ → Influencia de la heurística (distancia inversa).
  - $ Q $ → Cantidad de feromonas depositadas por cada hormiga.
  - $ \rho $ → Tasa de evaporación de feromonas.

#### **2. Construcción de Caminos**
Cada hormiga construye una solución siguiendo estos pasos:
- Comienza en un nodo aleatorio.
- Usa la función `add_nodo()` para **seleccionar el siguiente nodo** basado en:
  - **Feromonas en la arista**.
  - **Distancia inversa entre los nodos** (heurística).
- Continúa hasta visitar todos los nodos.
- Registra el camino y su longitud.

#### **3. Actualización de Feromonas**
- **Evaporación**: Se reducen las feromonas en todas las aristas (camino entre nodos).
- **Depósito de Feromonas**: Las hormigas depositan feromonas en las aristas que han recorrido:
  
  $
  	\tau_{ij} = (1 - \rho) \tau_{ij} + \sum_{k} \frac{Q}{L_k}
  $
  
  donde:
  - $ \tau_{ij} $ → Cantidad de feromonas en la arista $ (i, j) $.
  - $ \rho $ → Factor de evaporación.
  - $ Q $ → Cantidad de feromonas depositadas por cada hormiga.
  - $ L_k $ → Longitud total del recorrido de la hormiga $ k $.

#### **4. Selección de la Mejor Solución**
Después de varias iteraciones:
- Se elige el **mejor camino** registrado.
- Se devuelve la solución óptima encontrada.

In [ ]:
def colonia_hormigas(problem: tsplib95.models.Problem,
                     num_hormigas: int, 
                     num_iteraciones: int, 
                     alpha: float, beta: float, 
                     rho: float, Q: float) -> tuple:
    """
    Ejecuta el algoritmo de colonia de hormigas para resolver el TSP.

    Args:
    - problem (tsplib95.models.Problem): Problema del agente viajero.
    - num_hormigas (int): Número de hormigas a utilizar.
    - num_iteraciones (int): Número de iteraciones a ejecutar.
    - alpha (float): Peso de la influencia de las feromonas.
    - beta (float): Peso de la influencia de la heurística de distancia.
    - rho (float): Tasa de evaporación de feromonas.
    - Q (float): Cantidad de feromonas a depositar en cada iteración.

    Returns:
    tuple: Tupla con la mejor solución encontrada y su distancia total.

    """
    ## Inicializar feromonas y distancias
    nodos = list(problem.get_nodes())
    num_nodos = len(nodos)
    feromonas = np.ones((num_nodos, num_nodos)) / num_nodos # Donde cada nodo tiene 1/n feromonas
    distancias = np.array([[distancia(i, j, problem) for j in nodos] for i in nodos])
    
    mejor_solucion = None
    mejor_distancia = float('inf')
    
    for _ in range(num_iteraciones):
        soluciones = []
        
        for _ in range(num_hormigas):
            ## Para que cada hormiga inicie en un nodo aleatorio
            solucion = [np.random.choice(nodos)]

            ## Construir la solucion para cada hormiga
            while len(solucion) < num_nodos:
                nodos_disponibles = [n for n in nodos if n not in solucion]
                siguiente = add_nodo(solucion[-1], nodos_disponibles, nodos, feromonas, distancias, alpha, beta)
                ## Añadir el nodo seleccionado a la solucion
                solucion.append(siguiente)
            ## Añadir la solucion a la lista de soluciones
            soluciones.append(solucion)
        
        ## Calcular la distancia de cada solucion
        for solucion in soluciones:
            costo = distancia_total(solucion, problem)
            if costo < mejor_distancia:
                mejor_solucion, mejor_distancia = solucion, costo
        
        ## Actualizar feromonas
        feromonas *= (1 - rho)
        for solucion in soluciones:
            costo = distancia_total(solucion, problem)
            for i in range(num_nodos - 1):
                feromonas[solucion[i], solucion[i+1]] += Q / costo
    
    return mejor_solucion, mejor_distancia

In [6]:
problem = tsplib95.load('swiss42.tsp')
solucion, distancia = colonia_hormigas(problem, num_hormigas=100, num_iteraciones=100, alpha=1.0, beta=2.0, rho=0.5, Q=100)
print("Mejor solución encontrada:", solucion)
print("Distancia óptima:", distancia)

Mejor solución encontrada: [35, 36, 17, 31, 7, 37, 15, 16, 14, 19, 13, 5, 26, 18, 12, 11, 25, 10, 8, 9, 23, 41, 40, 24, 21, 39, 22, 38, 30, 29, 28, 27, 2, 3, 4, 6, 1, 0, 32, 34, 33, 20]
Distancia óptima: 1287
